In [1]:
import cv2
import glob
import os
import numpy as np

### Load sample video, extract a frame, and create a template 

In [2]:
video_path = '/media/yiting/NewVolume/Data/Videos/Camera_Alignment/2025-12-18/cameras/2025-12-18_11-22-05_062292/camTL-orig.mp4'
cap_base = cv2.VideoCapture(video_path)
cap_base.set(cv2.CAP_PROP_POS_FRAMES, 10)
ret, frame_base = cap_base.read()
cap_base.release()
gray_base = cv2.cvtColor(frame_base, cv2.COLOR_BGR2GRAY)
# Our camera setup: 1 pixel = 0.1-0.2 mm

# camTL ROI
x_base = 469
y_base = 481
w = 253
h = 142
# Create the template image
template = gray_base[y_base:y_base+h, x_base:x_base+w]

### Simulation functions- translation, rotation

In [ ]:
def shift_image(image, shift_x, shift_y, fill_color=(0, 0, 0)):
    """
    Shifts an image by a specific number of pixels in x and y directions.
    
    Args:
        image: The input image array (numpy array).
        shift_x: Pixels to shift horizontally (+ right, - left).
        shift_y: Pixels to shift vertically (+ down, - up).
        fill_color: BGR tuple to fill the new empty space (default black).
        
    Returns:
        The shifted image.
    """
    height, width = image.shape[:2]

    # --- The Math Behind the Shift ---
    # We create a 2x3 transformation matrix M.
    # The first two columns are an "identity matrix" (meaning keep scale and rotation the same), 
    # and the last column defines the translation (movement) in X and Y.
    T_matrix = np.float32([
        [1, 0, shift_x],
        [0, 1, shift_y]
    ])

    # Apply the affine transformation.
    # borderValue determines the color of the newly created empty space.
    shifted_img = cv2.warpAffine(
        image, 
        T_matrix, 
        (width, height), 
        borderMode=cv2.BORDER_CONSTANT, 
        borderValue=fill_color
    )

    return shifted_img

In [3]:
def rotate_image(image, angle, center=None, scale=1.0, fill_color=(0, 0, 0)):
    """
    Rotates an image by a specific angle.
    
    Args:
        image: Input image.
        angle: Rotation angle in degrees (Positive = Counter-Clockwise).
        center: (x, y) tuple for the center of rotation. None = Image Center.
        scale: Optional scaling factor (1.0 = keep original size).
        fill_color: Color to fill the empty corners (default black).
    """
    (h, w) = image.shape[:2]

    # If no center is defined, rotate around the center of the image
    if center is None:
        center = (w // 2, h // 2)

    # 1. Get the Rotation Matrix
    # This creates a 2x3 matrix for rotation + optional scaling
    M = cv2.getRotationMatrix2D(center, angle, scale)

    # 2. Apply the Rotation
    rotated = cv2.warpAffine(
        image, 
        M, 
        (w, h), 
        borderMode=cv2.BORDER_CONSTANT, 
        borderValue=fill_color
    )

    return rotated

### Translation Simulation

In [ ]:
# Define the shifts 
# +X = Right, -X = Left
# +Y = Down,  -Y = Up

# shifted_xs = np.arange(-20,20)
shifted_ys = np.arange(-20,20)
shifted_xs = [0]*len(shifted_ys)

shifted_imgs = []
labels = []

# Add original for comparison
shifted_imgs.append(gray_base)
labels.append("Original")

# Apply shifts
for tx, ty in zip(shifted_xs, shifted_ys):
    shifted = shift_image(gray_base, tx, ty, fill_color=(0, 0, 0))
    shifted_imgs.append(shifted)
    labels.append(f"Shift X:{tx:+d}, Y:{ty:+d}")

In [ ]:
for label, shifted_img in zip(labels, shifted_imgs):
    # 5. Template Matching
    # We search for the 'template' inside 'gray_curr'
    # TM_CCOEFF_NORMED is best for lighting invariance.
    res = cv2.matchTemplate(shifted_img, template, cv2.TM_CCOEFF_NORMED)
    
    # Get the location of the best match
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    # For TM_CCOEFF_NORMED, the best match is the maximum value (max_loc)
    # max_loc is (x, y) top-left corner
    x_curr, y_curr = max_loc

    # 6. Calculate Shift
    shift_x = x_curr - x_base
    shift_y = y_curr - y_base
    
    # Confidence score (0 to 1). If < 0.8, the match might be wrong (e.g. object blocked).
    confidence = max_val 

    status_x_str = f"{shift_x:+d} px".ljust(8)
    status_y_str = f"{shift_y:+d} px".ljust(8)

    print(f"{label} | {status_x_str} | {status_y_str} | {confidence:.2f}")

### Rotation simulation

In [6]:
# Angles to simulate (Degrees)
# Positive = Counter-Clockwise
# Negative = Clockwise
angles = np.arange(-20,20)

rotated_imgs = []
rotated_labels = []

for angle in angles:
    rotated = rotate_image(gray_base, angle, fill_color=(0, 0, 0))
    
    rotated_imgs.append(rotated)
    rotated_labels.append(f"{angle} degrees")

In [7]:
for rotated_label, rotated_img in zip(rotated_labels, rotated_imgs):
    # 5. Template Matching
    # We search for the 'template' inside the current image (rotated_img)
    # TM_CCOEFF_NORMED is best for lighting invariance.
    res = cv2.matchTemplate(rotated_img, template, cv2.TM_CCOEFF_NORMED)
    
    # Get the location of the best match
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    # For TM_CCOEFF_NORMED, the best match is the maximum value (max_loc)
    # max_loc is (x, y) top-left corner
    x_curr, y_curr = max_loc

    # 6. Calculate Shift
    shift_x = x_curr - x_base
    shift_y = y_curr - y_base
    
    # Confidence score (0 to 1). If < 0.8, the match might be wrong (e.g. object blocked).
    confidence = max_val 

    status_x_str = f"{shift_x:+d} px".ljust(8)
    status_y_str = f"{shift_y:+d} px".ljust(8)

    print(f"{rotated_label} | {status_x_str} | {status_y_str} | {confidence:.2f}")

-20 degrees | +25 px   | -21 px   | 0.82
-19 degrees | +23 px   | -19 px   | 0.82
-18 degrees | +22 px   | -19 px   | 0.83
-17 degrees | +21 px   | -18 px   | 0.84
-16 degrees | +20 px   | -17 px   | 0.85
-15 degrees | +18 px   | -16 px   | 0.86
-14 degrees | +17 px   | -15 px   | 0.87
-13 degrees | +16 px   | -14 px   | 0.88
-12 degrees | +14 px   | -13 px   | 0.90
-11 degrees | +13 px   | -12 px   | 0.91
-10 degrees | +12 px   | -11 px   | 0.92
-9 degrees | +10 px   | -10 px   | 0.93
-8 degrees | +9 px    | -9 px    | 0.94
-7 degrees | +8 px    | -8 px    | 0.95
-6 degrees | +7 px    | -7 px    | 0.96
-5 degrees | +5 px    | -6 px    | 0.97
-4 degrees | +4 px    | -5 px    | 0.98
-3 degrees | +3 px    | -4 px    | 0.99
-2 degrees | +2 px    | -2 px    | 0.99
-1 degrees | +1 px    | -1 px    | 1.00
0 degrees | +0 px    | +0 px    | 1.00
1 degrees | -1 px    | +1 px    | 1.00
2 degrees | -2 px    | +2 px    | 0.99
3 degrees | -3 px    | +4 px    | 0.98
4 degrees | -4 px    | +5 px    |